The main objective of the following article is to get a step-by-step guide on how to include exogenous variables and predict interval to our Time Series model using `Mlforecast`.

During this walkthrough, we will become familiar with the main `MlForecast` class and some relevant methods such as `mlforecast.fit`, `mlforecast.predict` and `mlforecast.cross_validation` in other.

Let's start!!!

<a class="anchor" id="0.1"></a>



1.	[Introduction](#1)
2.	[Loading libraries and data](#3)
3.	[Explore Data with the plot method](#3)
4.	[Training A Multivariate Time Series Model With MLForecast](#4)
5.  [Feature importances](#5)
6.  [Evaluate the model’s performance](#6)
7.  [Evaluate the model](#7)
8.  [References](#8)

# **Introduction** <a class="anchor" id=""></a>

[Table of Contents](#0)

Multivariate time series models are statistical or machine learning models used to analyze data sets consisting of multiple time series variables that are believed to be interrelated and influence each other over time. These models capture the complex relationships and dynamics between different variables and enable more accurate forecasting and analysis compared to univariate time series models. The models use machine learning algorithms to capture the complex relationships and dynamics between multiple time series variables.

Unlike traditional statistical models such as VAR, which are based on specific assumptions, multivariate time series models with machine learning have the advantage of being more flexible and capable of capturing non-linear patterns and complex dependencies between variables.

In a multivariate time series model, each dependent variable is modeled as a linear or nonlinear function of its past values and the past values of the other variables in the system. The central idea is that the variables in the system are considered endogenous, that is, they are influenced by other variables within the system rather than being independent of each other.

Multivariate time series models with machine learning offer the ability to model and forecast complex time series with multiple variables, allowing for greater flexibility and the ability to capture non-linear relationships. However, they may also require more data and greater preprocessing effort compared to traditional statistical models.

# **Loading libraries and data** <a class="anchor" id="3"></a>

[Table of Contents](#0)

In [1]:
# Handling and processing of Data
# ==============================================================================
import numpy as np
import pandas as pd

import scipy.stats as stats

# Handling and processing of Data for Date (time)
# ==============================================================================
import datetime
import datetime as dt
import time
from datetime import datetime, timedelta

# 
# ==============================================================================
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.seasonal import seasonal_decompose 
# 
# ==============================================================================


In [2]:
# Models
# ==============================================================================
import xgboost as xgb

# Mlforecast
# ==============================================================================
from mlforecast import MLForecast
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean
from window_ops.ewm import ewm_mean
from mlforecast.target_transforms import Differences

from mlforecast.utils import PredictionIntervals
from mlforecast.utils import generate_daily_series, generate_prices_for_series
from utilsforecast.plotting import plot_series

In [3]:
# Plot
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
plt.style.use('grayscale') # fivethirtyeight  grayscale  classic
plt.rcParams['lines.linewidth'] = 1.5
dark_style = {
    'figure.facecolor': '#212946',  #   #008080
    'axes.facecolor': '#212946',
    'savefig.facecolor': '#212946',
    'axes.grid': True,
    'axes.grid.which': 'both',
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False,
    'grid.color': '#000000',  #2A3459
    'grid.linewidth': '1',
    'text.color': '0.9',
    'axes.labelcolor': '0.9',
    'xtick.color': '0.9',
    'ytick.color': '0.9',
    'font.size': 12 }
plt.rcParams.update(dark_style)
# Define the plot size
# ==============================================================================

plt.rcParams['figure.figsize'] = (18,7)

# Hide warnings
# ==============================================================================
import warnings
warnings.filterwarnings("ignore")

## **Read data**

In [4]:
df=pd.read_csv("https://raw.githubusercontent.com/Naren8520/Serie-de-tiempo-con-Machine-Learning/main/Data/energy_dataset.csv", parse_dates=["time"], usecols=lambda column: column != 'Unnamed: 0')
df.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,20264.0,38.41,56.04


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 18 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   time                                         35064 non-null  datetime64[ns]
 1   generation biomass                           35064 non-null  float64       
 2   generation fossil brown coal/lignite         35064 non-null  float64       
 3   generation fossil gas                        35064 non-null  float64       
 4   generation fossil hard coal                  35064 non-null  float64       
 5   generation fossil oil                        35064 non-null  float64       
 6   generation hydro pumped storage consumption  35064 non-null  float64       
 7   generation hydro run-of-river and poundage   35064 non-null  float64       
 8   generation hydro water reservoir             35064 non-null  float64       


The input to MlForecast is always a data frame in long format with three columns: unique_id, ds and y:

* The `unique_id` (string, int or category) represents an identifier for the series.

* The `ds` (datestamp) column should be of a format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp.

* The `y` (numeric) represents the measurement we wish to forecast.

In [6]:
df["unique_id"]="1"
df=df.rename(columns={"time": "ds", "price actual": "y"})
df.head()

,ds,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load actual,price day ahead,y,unique_id
0,2015-01-01 00:00:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,25385.0,50.10,65.41,1
1,2015-01-01 01:00:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,24382.0,48.10,64.92,1
2,2015-01-01 02:00:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,22734.0,47.33,64.48,1
3,2015-01-01 03:00:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,21286.0,42.27,59.32,1
4,2015-01-01 04:00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,20264.0,38.41,56.04,1


# **Explore Data with the plot method** <a class="anchor" id="3"></a>

[Table of Contents](#0)

We are going to use the `plot_series` function to visualize our data.

In [32]:
fig = plot_series(df,max_insample_length=1000, palette="magma")
fig.savefig('../figs/multivariate_forecasting_and_prediction_intervals__eda.png')

![](../figs/multivariate_forecasting_and_prediction_intervals__eda.png)

## **The Augmented Dickey-Fuller Test**
An Augmented Dickey-Fuller (ADF) test is a type of statistical test that determines whether a unit root is present in time series data. Unit roots can cause unpredictable results in time series analysis. A null hypothesis is formed in the unit root test to determine how strongly time series data is affected by a trend. By accepting the null hypothesis, we accept the evidence that the time series data is not stationary. By rejecting the null hypothesis or accepting the alternative hypothesis, we accept the evidence that the time series data is generated by a stationary process. This process is also known as stationary trend. The values of the ADF test statistic are negative. Lower ADF values indicate a stronger rejection of the null hypothesis.

Augmented Dickey-Fuller Test is a common statistical test used to test whether a given time series is stationary or not. We can achieve this by defining the null and alternate hypothesis.

- Null Hypothesis: Time Series is non-stationary. It gives a time-dependent trend.
- Alternate Hypothesis: Time Series is stationary. In another term, the series doesn’t depend on time.

- ADF or t Statistic < critical values: Reject the null hypothesis, time series is stationary.
- ADF or t Statistic > critical values: Failed to reject the null hypothesis, time series is non-stationary.

In [8]:
def augmented_dickey_fuller_test(series , column_name):
    print (f'Dickey-Fuller test results for columns: {column_name}')
    dftest = adfuller(series, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','No Lags Used','Number of observations used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    if dftest[1] <= 0.05:
        print("Conclusion:====>")
        print("Reject the null hypothesis")
        print("The data is stationary")
    else:
        print("Conclusion:====>")
        print("The null hypothesis cannot be rejected")
        print("The data is not stationary")

In [9]:
augmented_dickey_fuller_test(df["y"],"Energy production")

Dickey-Fuller test results for columns: Energy production
Test Statistic                -9.147016e+00
p-value                        2.750493e-15
No Lags Used                   5.000000e+01
Number of observations used    3.501300e+04
Critical Value (1%)           -3.430537e+00
Critical Value (5%)           -2.861623e+00
Critical Value (10%)          -2.566814e+00
dtype: float64
Conclusion:====>
Reject the null hypothesis
The data is stationary


# **Training A Multivariate Time Series Model With MLForecast**<a class="anchor" id="4"></a>

[Table of Contents](#0)

## **Building Model**

Let’s see how we can engineer features and train an `XGBoost` with mlforecast.

In [10]:
model = [xgb.XGBRegressor(random_state=0, n_estimators=100)]

We fit the models by instantiating a new `MlForecast` object with the following parameters:

* `models:` a list of models. Select the models you want from models and import them.

* `freq:` a string indicating the frequency of the data. (See [panda’s available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).)

* `lags:` Lags of the target to uses as feature.

* `lag_transforms:` Mapping of target lags to their transformations.

* `date_features:` Features computed from the dates. Can be `pandas` date attributes or functions that will take the dates as input.

* `differences:` Differences to take of the target before computing the features. These are restored at the forecasting step.

* `num_threads:` Number of threads to use when computing the features.

* `target_transforms:` Transformations that will be applied to the target computing the features and restored after the forecasting step.

Any settings are passed into the constructor. Then you call its fit method and pass in the historical data frame.

In [11]:
from window_ops.rolling import rolling_mean, rolling_max, rolling_min

In [12]:
mlf = MLForecast(models=model,
                   freq='H',
                   lags=[1,2,4],
                   lag_transforms={
                       1: [(rolling_mean, 4), (rolling_min, 4), (rolling_max, 4)], 
                   },
                   date_features=['week', 'month','day'],
                   num_threads=6)

You can use any scikit-learn model, or any other model that follows the same API. We could use LightGBM or CatBoost, for example.

The MLForecast object will manage the transformation of the data, the training of the models, and the prediction of the target variable.

We pass the `freq` parameter to tell `mlforecast` that our data is hourly, and the `lags` parameter to tell it that we want to use the last 1, 2 and 4 hors as features to predict the next hours.

We also pass the `lag_transforms` parameter, which is a dictionary that maps the number of lags to a list of functions to apply to the lagged values.

The keys are the lags over which we want to apply the functions, and the values are lists of tuples, where the first element is the function to apply, and the second element is the number of weeks to apply it to (the window).

So in this case, we will apply the `rolling_mean, rolling_min` and `rolling_max` functions to the last 30 hours of each lag.

We are recommend we use the `window_ops` package or create your custom functions with numba.

date features is a list of the date components we want to use. These are features like the month, the week of the year, the day of the week (if it was a daily time series), etc.

Finally, `num_threads` is the number of threads to use when training the models in parallel. Setting it to the number of cores in your machine is a good idea.

We can use the `MLForecast.preprocess` method to explore different transformations.

In [13]:
prep = mlf.preprocess(df)
prep.head()

,ds,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,...,unique_id,lag1,lag2,lag4,rolling_mean_lag1_window_size4,rolling_min_lag1_window_size4,rolling_max_lag1_window_size4,week,month,day
4,2015-01-01 04:00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,...,1,59.32,64.48,65.41,63.5325,59.32,65.41,1,1,1
5,2015-01-01 05:00:00,410.0,178.0,4038.0,3590.0,156.0,2109.0,952.0,743.0,7098.0,...,1,56.04,59.32,64.92,61.1900,56.04,64.92,1,1,1
6,2015-01-01 06:00:00,401.0,172.0,4040.0,3368.0,158.0,2108.0,961.0,848.0,7098.0,...,1,53.63,56.04,64.48,58.3675,53.63,64.48,1,1,1
7,2015-01-01 07:00:00,408.0,172.0,4030.0,3208.0,160.0,2031.0,983.0,1012.0,7099.0,...,1,51.73,53.63,59.32,55.1800,51.73,59.32,1,1,1
8,2015-01-01 08:00:00,413.0,177.0,4052.0,3335.0,161.0,2119.0,1001.0,1015.0,7098.0,...,1,51.43,51.73,56.04,53.2075,51.43,56.04,1,1,1


In [14]:
prep.drop(columns=['unique_id', 'ds']).corr()['y']

generation biomass                             0.142659
generation fossil brown coal/lignite           0.364009
generation fossil gas                          0.461447
generation fossil hard coal                    0.465645
generation fossil oil                          0.285258
generation hydro pumped storage consumption   -0.426265
generation hydro run-of-river and poundage    -0.136663
generation hydro water reservoir               0.071858
generation nuclear                            -0.053030
generation other                               0.099919
generation other renewable                     0.255601
generation solar                               0.098572
generation waste                               0.168825
generation wind onshore                       -0.220444
total load actual                              0.435660
price day ahead                                0.732172
y                                              1.000000
lag1                                           0

## **Fit method**

Before we train the model, let’s create two lists with the names of the dynamic and static features.

* `Dynamic features` are the ones that change over time. You need to have these values for the future timestamps you want to predict.
* `Static features` are the ones that don’t change over time. MLForecast will repeat the first value of each static feature for all the timestamps you want to predict.

For this example we have considered some variables such as `Static features` although they are not necessarily constant or static over time, but for the purpose of exemplifying how to add exogenous variables to our model it is a good way to exemplify the use of the `Dynamic features` variables such as those of the `Static features`.

In [15]:
dynamic_features = ['generation fossil brown coal/lignite','generation fossil gas','generation fossil hard coal','generation hydro pumped storage consumption',
                    'generation hydro run-of-river and poundage','generation hydro water reservoir','generation solar','generation waste', 
                    'generation wind onshore', 'total load actual','price day ahead']

static_features = ['generation biomass','generation fossil oil','generation nuclear','generation other','generation other renewable']

The fit method uses the following parameters:
|Parameters |Type	|Default	|Details|
|-----------|-------|-----------|-------|
|df|	DataFrame|		|Series data in long format.|
|id_col|	str	|unique_id|	Column that identifies each serie|.
|time_col	|str	|ds	|Column that identifies each timestep, its values can be timestamps or integers.
|target_col	|str	|y	|Column that contains the target.
|static_features	|typing.Optional[typing.List[str]]|	None	|Names of the features that are static and will be repeated when forecasting. If None, will consider all columns (except id_col and time_col) as static.|
|dropna	|bool	|True	|Drop rows with missing values produced by the transformations.|
|keep_last_n	|typing.Optional[int]	|None	|Keep only these many records from each serie for the forecasting step. Can save time and memory if your features allow it.|
|max_horizon	|typing.Optional[int]	|None	|Train this many models, where each model will predict a specific horizon.|
|prediction_intervals	|typing.Optional[mlforecast.utils.PredictionIntervals]	|None	|Configuration to calibrate prediction intervals (Conformal Prediction).|
|fitted	|bool	|False	|Save in-sample predictions.|
|data	|typing.Optional[pandas.core.frame.DataFrame]	|None	|Series data in long format. This argument has been replaced by df and will be removed in a later release.|

In [16]:
mlf.fit(df, fitted=True, static_features=static_features,  
        prediction_intervals=PredictionIntervals(n_windows=5, window_size=30, method="conformal_distribution"))

MLForecast(models=[XGBRegressor], freq=<Hour>, lag_features=['lag1', 'lag2', 'lag4', 'rolling_mean_lag1_window_size4', 'rolling_min_lag1_window_size4', 'rolling_max_lag1_window_size4'], date_features=['week', 'month', 'day'], num_threads=6)

- Note that when we use the fit method, we include the `static_features` parameter.

- If you have no `static features`, pass an empty list or MLForecast will consider that all your additional columns are static.

- By default the forecast is generated in a recursive way, but we can also use the `max_horizon` parameter to generate the forecast with the direct method (one model for each step ahead);



In [ ]:
mlf.fit(df, static_features=static_features, max_horizon=30, fitted=True,
        prediction_intervals=PredictionIntervals(n_windows=5, window_size=30, method="conformal_distribution"))

- When using the `max_horizon` parameter in the fit method, this may take more than 1 minute, which will depend on each team.

* Let's see the results of our model in this case the `XGBoost model`. We can observe it with the following instruction:

Let us now visualize the fitted values of our models.

In [17]:
result=mlf.forecast_fitted_values()
result=result.set_index("unique_id")
result

,ds,y,XGBRegressor
unique_id,,,
1,2015-01-01 04:00:00,56.04,56.156284
1,2015-01-01 05:00:00,53.63,53.193878
1,2015-01-01 06:00:00,51.73,51.625008
1,2015-01-01 07:00:00,51.43,51.410889
1,2015-01-01 08:00:00,48.98,51.442638
...,...,...,...
1,2018-12-31 19:00:00,77.02,78.329239
1,2018-12-31 20:00:00,76.16,76.288696
1,2018-12-31 21:00:00,74.30,73.400200


In [18]:
from statsmodels.stats.diagnostic import normal_ad
from scipy import stats

sw_result = stats.shapiro(result["XGBRegressor"])
ad_result = normal_ad(np.array(result["XGBRegressor"]), axis=0)
dag_result = stats.normaltest(result["XGBRegressor"], axis=0, nan_policy='propagate')

It's important to note that we can only use this method if we assume that the residuals of our validation predictions are normally distributed. To see if this is the case, we will use a PP-plot and test its normality with the Anderson-Darling, Kolmogorov-Smirnov, and D’Agostino K^2 tests.

The PP-plot(Probability-to-Probability) plots the data sample against the normal distribution plot in such a way that if normally distributed, the data points will form a straight line.

The three normality tests determine how likely a data sample is from a normally distributed population using p-values. The null hypothesis for each test is that "the sample came from a normally distributed population". This means that if the resulting p-values are below a chosen alpha value, then the null hypothesis is rejected. Thus there is evidence to suggest that the data comes from a non-normal distribution. For this article, we will use an Alpha value of 0.01.

In [33]:
result=mlf.forecast_fitted_values()
fig, axs = plt.subplots(nrows=2, ncols=2)

# plot[1,1]
result["XGBRegressor"].plot(ax=axs[0,0])
axs[0,0].set_title("Residuals model");

# plot
#plot(result["XGBRegressor"], ax=axs[0,1]);
axs[0,1].hist(result["XGBRegressor"], density=True,bins=50, alpha=0.5 )
axs[0,1].set_title("Density plot - Residual");

# plot
stats.probplot(result["XGBRegressor"], dist="norm", plot=axs[1,0])
axs[1,0].set_title('Plot Q-Q')
axs[1,0].annotate("SW p-val: {:.4f}".format(sw_result[1]), xy=(0.05,0.9), xycoords='axes fraction', fontsize=15,
            bbox=dict(boxstyle="round", fc="none", ec="gray", pad=0.6))

axs[1,0].annotate("AD p-val: {:.4f}".format(ad_result[1]), xy=(0.05,0.8), xycoords='axes fraction', fontsize=15,
            bbox=dict(boxstyle="round", fc="none", ec="gray", pad=0.6))

axs[1,0].annotate("DAG p-val: {:.4f}".format(dag_result[1]), xy=(0.05,0.7), xycoords='axes fraction', fontsize=15,
            bbox=dict(boxstyle="round", fc="none", ec="gray", pad=0.6))
# plot
plot_acf(result["XGBRegressor"],  lags=35, ax=axs[1,1],color="fuchsia")
axs[1,1].set_title("Autocorrelation");

plt.savefig("../figs/multivariate_forecasting_and_prediction_intervals__plot_residual_model.png")
plt.close();

![](../figs/multivariate_forecasting_and_prediction_intervals__plot_residual_model.png)

## **Predict method with prediction intervals**

To generate forecasts use the predict method.

The predict method takes several arguments.: 

|Parameters |Type	|Default	|Details|
|-----------|-------|-----------|-------|
|h	|int	|	|Number of periods to predict.|
|dynamic_dfs	|typing.Optional[typing.List[pandas.core.frame.DataFrame]]	|None	|Future values of the dynamic features, e.g. prices.
|before_predict_callback	|typing.Optional[typing.Callable]	|None	|Function to call on the features before computing the predictions. This function will take the input dataframe that will be passed to the model for predicting and should return a dataframe with the same structure. The series identifier is on the index.
|after_predict_callback	|typing.Optional[typing.Callable]	|None	|Function to call on the predictions before updating the targets. This function will take a pandas Series with the predictions and should return another one with the same structure. The series identifier is on the index.
|new_df	|typing.Optional[pandas.core.frame.DataFrame]|	None	|Series data of new observations for which forecasts are to be generated. This dataframe should have the same structure as the one used to fit the model, including any features and time series data. If new_df is not None, the method will generate forecasts for the new observations.
|level	|typing.Optional[typing.List[typing.Union[int, float]]]|	None	|Confidence levels between 0 and 100 for prediction intervals.
|X_df	|typing.Optional[pandas.core.frame.DataFrame]	|None	|Dataframe with the future exogenous features. Should have the id column and the time column.
|ids	|typing.Optional[typing.List[str]]	|None	|List with subset of ids seen during training for which the forecasts should be computed.
|horizon	|typing.Optional[int]|	None	|Number of periods to predict. This argument has been replaced by h and will be removed in a later release.
|new_data	|typing.Optional[pandas.core.frame.DataFrame]	|None	|Series data of new observations for which forecasts are to be generated. This dataframe should have the same structure as the one used to fit the model, including any features and time series data. If new_data is not None, the method will generate forecasts for the new observation



The forecast object here is a new data frame that includes a column with the name of the model and the y hat values, as well as columns for the uncertainty intervals.

This step should take less than 1 second.

In [20]:
forecast_df =mlf.predict(horizon=30,level=[80,95],dynamic_dfs=[df[['unique_id','ds']+dynamic_features]])
forecast_df


,unique_id,ds,XGBRegressor,XGBRegressor-lo-95,XGBRegressor-lo-80,XGBRegressor-hi-80,XGBRegressor-hi-95
0,1,2019-01-01 00:00:00,67.201607,66.026950,66.427206,67.976008,68.376263
1,1,2019-01-01 01:00:00,64.311119,62.169308,62.311854,66.310384,66.452930
2,1,2019-01-01 02:00:00,63.003006,60.632687,60.703998,65.302014,65.373325
3,1,2019-01-01 03:00:00,61.003784,58.959198,59.253872,62.753696,63.048370
4,1,2019-01-01 04:00:00,60.615204,58.270322,58.458266,62.772141,62.960086
5,1,2019-01-01 05:00:00,61.224861,58.845230,59.680048,62.769674,63.604492
6,1,2019-01-01 06:00:00,62.603184,61.060147,61.498018,63.708350,64.146221
7,1,2019-01-01 07:00:00,65.311356,58.622908,61.759400,68.863311,71.999803
8,1,2019-01-01 08:00:00,69.205391,65.525402,66.567001,71.843781,72.885379
9,1,2019-01-01 09:00:00,71.437088,69.869902,70.025982,72.848194,73.004274


In [34]:
fig = plot_series(df, forecast_df, level=[80,95], max_insample_length=200,engine="matplotlib", palette="magma")
fig.get_axes()[0].set_title("Prediction with intervals")
fig.savefig('../figs/multivariate_forecasting_and_prediction_intervals__plot_forecasting_intervals.png')

![](../figs/multivariate_forecasting_and_prediction_intervals__plot_forecasting_intervals.png)

# **Feature importances** <a class="anchor" id="5"></a>

[Table of Contents](#0)

Take a look at the feature importances too.

This model is heavily dependent on the lag 1 feature, as you can see in the feature importance plot for XGBRegressor.

In [35]:
fig=pd.Series(mlf.models_['XGBRegressor'].feature_importances_, 
              index=mlf.ts.features_order_).sort_values(ascending=False).plot.bar(title='Feature Importance XGBRegressor')
plt.savefig('../figs/multivariate_forecasting_and_prediction_intervals__plot_feature_importance.png',dpi=300)
plt.close()

![](../figs/multivariate_forecasting_and_prediction_intervals__plot_feature_importance.png)

# **Evaluate the model’s performance** <a class="anchor" id="6"></a>

[Table of Contents](#0.1)

In previous steps, we’ve taken our historical data to predict the future. However, to asses its accuracy we would also like to know how the model would have performed in the past. To assess the accuracy and robustness of your models on your data perform Cross-Validation.

With time series data, Cross Validation is done by defining a sliding window across the historical data and predicting the period following it. This form of cross-validation allows us to arrive at a better estimation of our model’s predictive abilities across a wider range of temporal instances while also keeping the data in the training set contiguous as is required by our models.

The following graph depicts such a Cross Validation Strategy:

![](https://raw.githubusercontent.com/Nixtla/statsforecast/main/nbs/imgs/ChainedWindows.gif)

## **Perform time series cross-validation**

In order to get an estimate of how well our model will be when predicting future data we can perform cross validation, which consist on training a few models independently on different subsets of the data, using them to predict a validation set and measuring their performance.

Since our data depends on time, we make our splits by removing the last portions of the series and using them as validation sets. This process is implemented in `MLForecast.cross_validation`.

Cross-validation of time series models is considered a best practice but most implementations are very slow. The `MlForecast` library implements cross-validation as a distributed operation, making the process less time-consuming to perform. If you have big datasets you can also perform Cross Validation in a distributed cluster using `Ray, Dask or Spark`.

Depending on your computer, this step should take around 1 min.

The cross_validation method from the StatsForecast class takes the following arguments.

* `df:` training data frame

* `h (int):` represents h steps into the future that are being forecasted. In this case, 12 months ahead.

* `step_size (int):` step size between each window. In other words: how often do you want to run the forecasting processes.

* `n_windows(int):` number of windows used for cross validation. In other words: what number of forecasting processes in the past do you want to evaluate.

In [23]:
cv_result = mlf.cross_validation(
    df,
    n_windows=5,  # number of models to train/splits to perform
    window_size=30,  # length of the validation set in each window
    static_features=static_features,
    prediction_intervals=PredictionIntervals(n_windows=5, window_size=30, method="conformal_distribution")
)

The crossvaldation_df object is a new data frame that includes the following columns:

* `unique_id:` index. If you dont like working with index just run `crossvalidation_df.resetindex()`.
* `ds:` datestamp or temporal index
* `cutoff:` the last datestamp or temporal index for the `n_windows`.
* `y:` true value
* `model:` columns with the model’s name and fitted value.

In [24]:
cv_result

,unique_id,ds,cutoff,y,XGBRegressor
0,1,2018-12-25 18:00:00,2018-12-25 17:00:00,71.73,72.156570
1,1,2018-12-25 19:00:00,2018-12-25 17:00:00,71.94,73.250244
2,1,2018-12-25 20:00:00,2018-12-25 17:00:00,73.05,73.350319
3,1,2018-12-25 21:00:00,2018-12-25 17:00:00,73.94,74.397888
4,1,2018-12-25 22:00:00,2018-12-25 17:00:00,71.98,73.627937
...,...,...,...,...,...
25,1,2018-12-31 19:00:00,2018-12-30 17:00:00,77.02,73.463821
26,1,2018-12-31 20:00:00,2018-12-30 17:00:00,76.16,72.620155
27,1,2018-12-31 21:00:00,2018-12-30 17:00:00,74.30,71.002312
28,1,2018-12-31 22:00:00,2018-12-30 17:00:00,69.89,69.561783


# **Evaluate the model** <a class="anchor" id="7"></a>

[Table of Contents](#0.1)

We can now compute the accuracy of the forecast using an appropiate accuracy metric. Here we’ll use the Root Mean Squared Error (RMSE). To do this, we first need to `install datasetsforecast`, a Python library developed **by Nixtla** that includes a function to compute the RMSE.

`pip install datasetsforecast`

In [27]:
from datasetsforecast.losses import rmse

The function to compute the RMSE takes two arguments:

1. The actual values.
2. The forecasts, in this case, `XGBRegressor() Model`.

In [28]:
from datasetsforecast.losses import mse, mae, rmse

def evaluate_cross_validation(df, metric):
    models = df.drop(columns=['ds', 'cutoff', 'y']).columns.tolist()
    evals = []
    for model in models:
        eval_ = df.groupby(['unique_id', 'cutoff']).apply(lambda x: metric(x['y'].values, x[model].values)).to_frame() # Calculate loss for every unique_id, model and cutoff.
        eval_.columns = [model]
        evals.append(eval_)
    evals = pd.concat(evals, axis=1)
    evals = evals.groupby(['unique_id']).mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals

In [29]:
evaluation_df = evaluate_cross_validation(cv_result.set_index("unique_id"), rmse)

evaluation_df

,XGBRegressor,best_model
unique_id,,
1,1.81789,XGBRegressor


In [31]:
cv_rmse = cv_result.groupby(['unique_id', 'cutoff']).apply(lambda df: rmse(df['y'], df['XGBRegressor'])).mean()
print("RMSE using cross-validation: ", cv_rmse)

RMSE using cross-validation:  1.81789025583616


# **References** <a class="anchor" id="8"></a>

[Table of Contents](#0)


1. Changquan Huang • Alla Petukhina. Springer series (2022). Applied Time Series Analysis and Forecasting with Python. 
2. Ivan Svetunkov. [Forecasting and Analytics with the Augmented Dynamic Adaptive Model (ADAM)](https://openforecast.org/adam/)
3. [James D. Hamilton. Time Series Analysis Princeton University Press, Princeton, New Jersey, 1st Edition, 1994.](https://press.princeton.edu/books/hardcover/9780691042893/time-series-analysis)
4. [Nixtla Parameters for Mlforecast](https://nixtla.github.io/mlforecast/forecast.html).
5. [Pandas available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
6. [Rob J. Hyndman and George Athanasopoulos (2018). “Forecasting principles and practice, Time series cross-validation”.](https://otexts.com/fpp3/tscv.html).
7. [Seasonal periods- Rob J Hyndman](https://robjhyndman.com/hyndsight/seasonal-periods/).